In [1]:
pip install datasets evaluate transformers[torch] rouge-score nltk

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=e0b98dbba3f8bff67cd64c03f710a309c920c9f4d08139c5f1ca8299139b8797
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [4]:
import transformers

print(transformers.__version__)

4.35.0


In [5]:
model_checkpoint = "t5-small"

In [6]:
import pandas as pd
import nltk
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
summary = pd.read_csv('/kaggle/input/news-summary/news_summary.csv', encoding='iso-8859-1')
raw = pd.read_csv('/kaggle/input/news-summary/news_summary_more.csv', encoding='iso-8859-1')

In [16]:
pre1 =  raw.iloc[:,0:2].copy()

pre2 = summary.iloc[:,0:6].copy()
pre2['text'] = pre2['author'].str.cat(pre2['date'].str.cat(pre2['read_more'].str.cat(pre2['text'].str.cat(pre2['ctext'], sep = " "), sep =" "),sep= " "), sep = " ")

In [17]:
pre = pd.DataFrame()
pre['text'] = pd.concat([pre1['text'], pre2['text']], ignore_index=True)
pre['summary'] = pd.concat([pre1['headlines'],pre2['headlines']],ignore_index = True)

In [18]:
# Replace empty strings with NaN
pre.replace("", np.nan, inplace=True)

# Drop rows with any NaN values in 'text' or 'summary' columns
pre.dropna(subset=['text', 'summary'], inplace=True)

In [19]:
# Calculate the number of data points that correspond to 10%
number_of_rows = int(len(pre) * 0.4)

# Take the first 10% of the dataset
pre = pre.iloc[:number_of_rows]

In [20]:
pre.head(2)

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...


In [21]:
from evaluate import load
metric = load("rouge")

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [23]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [24]:
max_input_length = 1024
max_target_length = 128

In [25]:
def preprocess_function(examples):
    inputs = [prefix + (doc if doc is not None else "") for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    summaries = [summary if summary is not None else "" for summary in examples["summary"]]
    labels = tokenizer(text_target=summaries, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
from datasets import Dataset

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
train_df, validation_df = train_test_split(pre, test_size=0.1)

In [29]:
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [30]:
# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True)

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [31]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [32]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-news",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to="none"
)

In [33]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [34]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [35]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.996800,1.728400,47.232400,24.643800,43.181800,43.169700,15.847000


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2313, training_loss=2.1190038038650427, metrics={'train_runtime': 324.0603, 'train_samples_per_second': 114.195, 'train_steps_per_second': 7.138, 'total_flos': 1108138490855424.0, 'train_loss': 2.1190038038650427, 'epoch': 1.0})

In [37]:
print(f"Training Dataset Length: {len(tokenized_train_dataset)}")
print(f"Validation Dataset Length: {len(tokenized_validation_dataset)}")

Training Dataset Length: 37006
Validation Dataset Length: 4112


In [38]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

'https://huggingface.co/viktor-shevchuk/t5-small-finetuned-news/tree/main/'

In [39]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("viktor-shevchuk/t5-small-finetuned-news")

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [41]:
def generate_summary(text, max_length=150):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=max_length, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [42]:
num_examples_to_generate = 10
for i in range(num_examples_to_generate):
    text_to_summarize = pre.iloc[i]['text']
    generated_summary = generate_summary(text_to_summarize)
    original_summary = pre.iloc[i]['summary']
    print(f"Text: {text_to_summarize}")
    print(f"Original Summary: {original_summary}")
    print(f"Generated Summary: {generated_summary}\n")

Text: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.
Original Summary: upGrad learner switches to career in ML & Al with 90% salary hike
Generated Summary: UpGrad alum Saurav Kant was a Sr Systems Engineer at Infosys with 5 years of work experience. UpGrad's Online Power Learning has powered 3 lakh+ careers

Text: Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.
Origina

## Bart based model (BERT+GPT) with an added optimizer and learning rate scheduler:

In [43]:
bart_model_checkpoint = "facebook/bart-large-cnn"

In [44]:
bart_tokenizer = AutoTokenizer.from_pretrained(bart_model_checkpoint)

In [45]:
def bart_preprocess_function(examples):
    inputs = [(doc if doc is not None else "") for doc in examples["text"]]
    model_inputs = bart_tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    summaries = [summary if summary is not None else "" for summary in examples["summary"]]
    labels = bart_tokenizer(text_target=summaries, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [46]:
# Tokenize the datasets
bart_tokenized_train_dataset = train_dataset.map(bart_preprocess_function, batched=True)
bart_tokenized_validation_dataset = validation_dataset.map(bart_preprocess_function, batched=True)

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [47]:
bart_model = AutoModelForSeq2SeqLM.from_pretrained(bart_model_checkpoint)

In [48]:
batch_size = 16  # I can lower the batch size from 16 to less, which prevents the "out of memory" error
bart_model_name = bart_model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{bart_model_name}-finetuned-news",
    evaluation_strategy = "epoch",
    learning_rate=3e-5, # tweaked the learning rate slightly
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.05, # tweaked a bit to penalize large weights in the model. The model is encouraged to keep the weights small, leading to a simpler and more generalizable model
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to="none"
)

In [49]:
bart_data_collator = DataCollatorForSeq2Seq(bart_tokenizer, model=bart_model)

In [50]:
def bart_compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = bart_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, bart_tokenizer.pad_token_id)
    decoded_labels = bart_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != bart_tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Adding a custom optimizer can give more control over the training process. The AdamW optimizer is commonly used with transformer models.

In [51]:
from transformers import AdamW

bart_optimizer = AdamW(bart_model.parameters(), lr=5e-5, eps=1e-8, weight_decay=0.01)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


A learning rate scheduler can help in adjusting the learning rate during training, which can lead to better results. For instance, we can use a scheduler that reduces the learning rate once learning stagnates.

In [52]:
from transformers import get_scheduler

num_training_steps = len(bart_tokenized_train_dataset) * args.num_train_epochs
bart_lr_scheduler = get_scheduler(
    "linear",
    optimizer=bart_optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [53]:
bart_trainer = Seq2SeqTrainer(
    bart_model,
    args,
    train_dataset=bart_tokenized_train_dataset,
    eval_dataset=bart_tokenized_validation_dataset,
    data_collator=bart_data_collator,
    tokenizer=bart_tokenizer,
    compute_metrics=bart_compute_metrics,
    optimizers=(bart_optimizer, bart_lr_scheduler)  # custom optimizer and LR scheduler
)

In [55]:
bart_trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.814900,1.314022,43.349500,23.611700,38.150100,38.257500,57.970100


TrainOutput(global_step=2313, training_loss=0.7994637759382262, metrics={'train_runtime': 1911.6052, 'train_samples_per_second': 19.359, 'train_steps_per_second': 1.21, 'total_flos': 8121300683685888.0, 'train_loss': 0.7994637759382262, 'epoch': 1.0})

In [56]:
bart_trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

'https://huggingface.co/viktor-shevchuk/bart-large-cnn-finetuned-news/tree/main/'

In [57]:
bart_model = AutoModelForSeq2SeqLM.from_pretrained("viktor-shevchuk/bart-large-cnn-finetuned-news")

In [58]:
def bart_generate_summary(text, max_length=150):
    inputs = bart_tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = bart_model.generate(inputs, max_length=max_length, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [59]:
num_examples_to_generate = 10
for i in range(num_examples_to_generate):
    text_to_summarize = pre.iloc[i]['text']
    generated_summary = bart_generate_summary(text_to_summarize)
    original_summary = pre.iloc[i]['summary']
    print(f"Text: {text_to_summarize}")
    print(f"Original Summary: {original_summary}")
    print(f"Generated Summary: {generated_summary}\n")

Text: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.
Original Summary: upGrad learner switches to career in ML & Al with 90% salary hike
Generated Summary: UpGrad alumnus Saurav Kant was a Sr Systems Engineer at Infosys   Â  Ã¢ÂÂs upGrad program

Text: Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.
Original Summary: Delhi techie wins free food from Swiggy for one ye